<div style="border:solid blue 2px; padding: 20px">
<font color='blue'>Привет, Миша!</font>

Твоё решение проверил <font color='blue'>Александр Цымбалов</font>

Если у тебя будут какие-то <font color='blue'>вопросы</font>, то обязательно их <font color='blue'>задавай</font>

Я буду обращаться к тебе на "ты", ведь все мы здесь студенты)
Но если тебе некомфортно, то дай знать, пожалуйста
<br><br>
Мои комментарии будут в отдельных ячейках <font color='blue'>Markdown</font> с заголовком «Комментарий ревьюера».

**Пожалуйста, не перемещай, не изменяй и не удаляй их**. 
Я буду использовать цветовую разметку:

<div class="alert alert-danger">
<b>Комментарий ревьюера ❌:</b> Замечания, которые необходимо обработать, иначе я не смогу принять проект </div>

<div class="alert alert-warning">
<b>Комментарий ревьюера ⚠️:</b> Небольшие замечания или вопросы по пониманию темы. Если есть желание и время, то у тебя есть возможность сделать проект ещё лучше </div>

<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> Хвалебные отзывы или другого рода комментарии)</div>

**Если ты что-то меняешь в проекте или отвечаешь на мои комментарии — пиши об этом.**

Мне будет <font color='blue'> легче </font> отследить изменения, если ты выделишь свои комментарии:
<div class="alert alert-info"> <b>Комментарий студента:</b> Лучше в таком формате, но как тебе <font color='blue'> удобно </font></div></font>

Версии комментариев я буду нумеровать

<br>

<div style="border:solid blue 2px; padding: 20px">
<font color='blue'>Итоги 1 ревью</font>

Порог в работе не пройден, но основные действия по работе с BERT и обучение моделей произведены в большей части корректно. Что я могу предложить, чтобы увеличить показатели по метрике:
    <ul>
        <li>Использовать для обучения Google Colab</li>
        <li>Увеличить выборку (см про Google colab)</li>
        <b><li>Обязательно очистить текст перед работой (можно без лемматизации)</li></b>
        <li>Подобрать гиперпараметры для моделей</li>
        <li>Не масштабировать эмбеддинги</li>
    </ul>
Также хочу заметить, что TF-IDF/word2vec + модели классического ML тоже неплохо справляются с задачей. 
    <br><br>
    
<font color='blue'>Материалы, которые помогут разобраться:</font>

https://huggingface.co/transformers/model_doc/bert.html \
https://t.me/renat_alimbekov \
https://colah.github.io/posts/2015-08-Understanding-LSTMs/ - Про LSTM \
https://web.stanford.edu/~jurafsky/slp3/10.pdf - про энкодер-декодер модели, этеншены\
https://pytorch.org/tutorials/beginner/transformer_tutorial.html - официальный гайд
по трансформеру от создателей pytorch\
https://transformer.huggingface.co/ - поболтать с трансформером \
Библиотеки: allennlp, fairseq, transformers, tensorflow-text — множествореализованных
методов для трансформеров методов NLP \
Word2Vec https://radimrehurek.com/gensim/models/word2vec.html 

<font color='green'>Пример BERT с GPU:
```python
%%time
from tqdm import notebook
batch_size = 2 # для примера возьмем такой батч, где будет всего две строки датасета
embeddings = [] 
for i in notebook.tqdm(range(input_ids.shape[0] // batch_size)):
        batch = torch.LongTensor(input_ids[batch_size*i:batch_size*(i+1)]).cuda() # закидываем тензор на GPU
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).cuda()
        
        with torch.no_grad():
            model.cuda()
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy()) # перевод обратно на проц, чтобы в нумпай кинуть
        del batch
        del attention_mask_batch
        del batch_embeddings
        
features = np.concatenate(embeddings) 
```
Можно сделать предварительную проверку на наличие GPU.\
Например, так: ```device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")```\
Тогда вместо .cuda() нужно писать .to(device)</font><br><br>
Жду на ревью. Удачи!

# Проект для «Викишоп» c BERT

<div class="alert alert-success">
<b>1 Комментарий ревьюера ✔️:</b> Здорово, что ты взял BERT

### Описание проекта
Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.
Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.
Постройте модель со значением метрики качества F1 не меньше 0.75.


<div class="alert alert-success">
<b>1 Комментарий ревьюера ✔️:</b> Хорошая практика вставлять описание работы для того, чтобы видеть задачу перед глазами

In [108]:
# !pip install transformers

In [107]:
import numpy as np
import pandas as pd
import torch
import transformers
import spacy
from tqdm import notebook
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

from sklearn.metrics import f1_score

import os
import re
from lightgbm import LGBMClassifier

from sklearn.pipeline import Pipeline

**Доступность карты**

In [109]:
# Проверяю, доступна ли видеокарта
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [110]:
model_class, tokenizer_class, pretrained_weights = (transformers.DistilBertModel,
                                                    transformers.DistilBertTokenizer,
                                                    'distilbert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights).to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [111]:
# def get_data(filename):
    
#     """
#     Reads the data from the given path.
#     """

#     pth1 = f'/datasets/{filename}'
#     pth2 = f'data/{filename}'

#     if os.path.exists(pth1):
#         df = pd.read_csv(pth1)
#         print(f'Reading {pth1}...')
#     elif os.path.exists(pth2):
#         df = pd.read_csv(pth2)
#         print(f'Reading {pth2}...')
#     else:
#         print('Check the file path')

#     return df

### Загрузка данных

In [112]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

In [115]:
# df = get_data('toxic_comments.csv')
# Эмбеддинги буду получать для подвыборки (иначе слишком долго)
df = df.sample(50000, random_state=42).reset_index(drop=True)

df.head()

,text,toxic,clear_text
0,""" March 2006 (UTC)\n\nActually, I gave two, bo...",0,march utc actually i gave two both of which ar...
1,"To whom it may concern \nThis is a talk page, ...",0,to whom it may concern this is a talk page not...
2,"why is this front page news \n\njust asking, ...",0,why is this front page news just asking nobody...
3,==\nI IDOLIZE YOU MAN!!!,0,i idolize you man
4,"""\n\n Some baklava for you! \n\n A Gift, in t...",0,some baklava for you a gift in the hopes that ...


In [116]:
df.shape

(50000, 3)

In [117]:
df['toxic'].value_counts(normalize=True)

0    0.89814
1    0.10186
Name: toxic, dtype: float64

> В обучающей выборке существенный дисбаланс классов

**Очистка текста**

<div class="alert alert-info">
<b>Комментарий студента:</b>
Очистил текст
</font></div></font>

In [118]:
nlp = spacy.load('en_core_web_sm')

In [123]:
def clear_text(text):
    '''
    Text cleaning
    '''
    
    text_patterned = re.sub(r'[^a-zA-Z]', ' ', text.lower())
    text_splitted = text_patterned.split()
    text_spaced = ' '.join(text_splitted)
    
    return text_spaced

In [124]:
# def lemmatize(text):
#     global nlp
#     doc = nlp(text)
#     return " ".join([token.lemma_ for token in doc])

In [125]:
df['clear_text'] = df['text'].apply(clear_text)

df['clear_text']

0        march utc actually i gave two both of which ar...
1        to whom it may concern this is a talk page not...
2        why is this front page news just asking nobody...
3                                        i idolize you man
4        some baklava for you a gift in the hopes that ...
                               ...                        
49995    a little much is it really necessary to mentio...
49996    i have to disagree i will not be changing my w...
49997    could the name have something to do with the p...
49998    the perquot were used as slaves or traded as s...
49999    list of managers hey thanks for the warning i ...
Name: clear_text, Length: 50000, dtype: object

In [122]:
# df['clear_text'].apply(lemmatize)

### Токенизация и подготовка данных

In [126]:
tokenized = df['clear_text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

In [ ]:
# Извлечение эмбеддингов
batch_size = 200
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]).to(device)
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).to(device)
    
    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)
    
    embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())
    
    del batch
    del attention_mask_batch
    del batch_embeddings

In [ ]:
features = np.concatenate(embeddings)

features.shape

In [ ]:
# # Сохраняю эмбеддинги для запуска проекта локально
# with open('embeddings_50000.npy', 'wb') as f:
#     np.save(f, embeddings)
# with open('embeddings_50000.npy', 'rb') as f:
#     a = np.load(f)
# print(a)

<div class="alert alert-success">
<b>1 Комментарий ревьюера ✔️:</b> Правильная работа с BERT

In [ ]:
# features = np.concatenate(embeddings)
targets = df['toxic']

train_features, test_features, train_target, test_target = train_test_split(features, targets,
                                                                            test_size=.25,
                                                                            random_state=42)

<div class="alert alert-warning">
<b>1 Комментарий ревьюера ⚠️:</b> На курсе принято делить выборку в пропорциях 75% train 25% test. Кажется, что 25% весьма существенны для обучения</div>

**Исправил**

### Обучение моделей

<div class="alert alert-warning">
<b>1 Комментарий ревьюера ⚠️:</b> Поиск дисбаланса можно было произвести при первоначальном анализе данных</div>

**Перенес**

### catboost

In [ ]:
# !pip install catboost

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
cat = CatBoostClassifier(random_state=12345, task_type='GPU')
cat.fit(pd.DataFrame(train_features), train_target, verbose=100)

ValueError: ignored

In [ ]:
cat_pred = cat.predict(pd.DataFrame(test_features))
print(f1_score(cat_pred, test_target))

0.7069681587448085


#### Линейная регрессия

In [ ]:
lg_model = LogisticRegression(max_iter=500, class_weight='balanced')
lg_model.fit(train_features, train_target)

train_predict = lg_model.predict(train_features)
print(f'train F1: {f1_score(train_target, train_predict):.2f}')

test_predict = lg_model.predict(test_features)
print(f'test F1: {f1_score(test_target, test_predict):.2f}')

train F1: 0.69
test F1: 0.65


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
lg_CV_model = LogisticRegressionCV(cv=50, scoring='f1', max_iter=500, class_weight='balanced')
lg_CV_model.fit(train_features, train_target)

train_predict = lg_CV_model.predict(train_features)
print(f'train F1: {f1_score(train_target, train_predict):.2f}')

test_predict = lg_CV_model.predict(test_features)
print(f'test F1: {f1_score(test_target, test_predict):.2f}')

> Логистическая регрессия дает результат для метрики F1 на 0.1 ниже требуемого порогового

#### LGBM

In [ ]:
model = LGBMClassifier(class_weight='balanced', n_estimators=500)
# Оценка качества модели
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, train_features, train_target, scoring='f1', cv=cv, n_jobs=-1)
# report performance
print(f'F1: {np.mean(n_scores):.2f}')

<div class="alert alert-block alert-warning">
<b> Комментарий студента</b>
    
В обсуждении прочитал, что требуется использовать пайплан, так как в противном случае на тренировочных данных будет пременяться векторизатор, обученный на всех исходных данных (test + train). Я так и не понял, как это реализовать, но пайплайны построить попробовал:

</div>

<div class="alert alert-danger">
<b>1 Комментарий ревьюера ❌:</b> Я не думаю, что уместно для эмбеддингов BERT использовать масштабирование, т.к. они уже сами по себе в некотором роде отмасштабированы. Не стоит здесь использовать StandardScaler

In [ ]:
# В обсуждении прочитал, что требуется использовать пайплан, так как
# векторизатор обучается на всей выборке
pipeline = Pipeline([
    ('logreg', LogisticRegression(max_iter=500, class_weight='balanced'))
])

# Fit pipeline on training data
pipeline.fit(train_features, train_target)
pipeline_predictions = pipeline.predict(test_features)

# Score pipeline on testing data
print(f'test F1: {f1_score(test_target, pipeline_predictions):.2f}')

In [ ]:
C = np.logspace(-4, 4, 5)

C

array([1.e-04, 1.e-02, 1.e+00, 1.e+02, 1.e+04])

In [ ]:
# Create the pipeline object
# Note this is identical to the code above
polynomial_pipeline = Pipeline([
    ('logreg', LogisticRegressionCV(cv=20, scoring='f1', max_iter=500, class_weight='balanced'))
])

# Create new parameter dictionary
grid_params = {
    'logreg__penalty': ('l1', 'l2'),
    'C': np.logspace(-4, 4, 5),
}

# Instantiate new gridsearch object
gs_2 = GridSearchCV(polynomial_pipeline, grid_params)

# Fit model to our training data
gs_2.fit(train_features, train_target)
gs2_predictions = gs_2.predict(test_features)

# Score pipeline on testing data
print(f'test F1: {f1_score(test_target, gs2_predictions):.2f}')

> Все другие реализованные методы даюр результат хуже, чем логистическая регрессия. 

<div class="alert alert-block alert-warning">
<b> Комментарий студента</b>
    
Я явно упускаю что-то важное, но попробую разобраться с твоими подсказками.

</div>

<div class="alert alert-warning">
<b>1 Комментарий ревьюера ⚠️:</b> См. итоги 1 ревью